In [ ]:
import os 
print(%pwd)
os.chdir("../")
print(%pwd)

# Data Ingestion Entity Class 

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path 

# Configuration Manager Class

In [ ]:
from cnnClassifier.constants import CONFIG_PATH_YAML,PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml,create_directories
from cnnClassifier import logger

class ConfigurationManager:
    """
    Central manager responsible for:
    - Reading YAML configuration and parameter files
    - Creating required artifact directories
    - Providing strongly-typed config objects to pipeline components
    """

    def __init__(
        self,
        config_filepath=CONFIG_PATH_YAML,
        params_filepath=PARAMS_FILE_PATH
    ):
        # Load main configuration (paths, URLs, pipeline structure)
        self.config = read_yaml(config_filepath)

        # Load parameters configuration (hyperparameters, constants, etc.)
        self.params = read_yaml(params_filepath)

        # Ensure the root artifacts directory exists
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Creates and returns a DataIngestionConfig object
        containing all settings required for the data ingestion stage.
        """
        # Extract data ingestion section from main config
        config = self.config.data_ingestion

        # Create directory to store ingestion-related artifacts
        create_directories([config.root_dir])

        # Map YAML configuration to a structured config dataclass
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


# Data Ingestion Pipeline 

In [ ]:
import gdown
import zipfile
from cnnClassifier import logger

class DataIngestion:
    def __init__(self,config : DataIngestionConfig):
        self.config = config 
    
    def download_file(self)->str:

        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file

            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into {zip_download_dir}")
        except Exception as e:
            raise
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [49]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 

[2026-02-04 00:09:29,630: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2026-02-04 00:09:29,636: INFO: common: yaml file: params.yaml loaded successfully:]
[2026-02-04 00:09:29,638: INFO: common: created directory at: artifacts:]
[2026-02-04 00:09:29,641: INFO: common: created directory at: artifacts/data_ingestion:]
